In [7]:
pip install scikit-learn

In [8]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 33.9 MB/s eta 0:00:00


In [9]:
from datasets import load_dataset

imdb_dataset = load_dataset("imdb")['train']

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [10]:
train_data = []
train_data_labels = []
for item in imdb_dataset:
  train_data.append(item['text'])
  train_data_labels.append(item['label'])

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer='word',max_features=1000,lowercase=True,stop_words='english',ngram_range=(1,2))
features = vectorizer.fit_transform(train_data).toarray()

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(features,train_data_labels,train_size=0.75,random_state=123)

In [13]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
model_nb = MultinomialNB()
model_dt = DecisionTreeClassifier()
model_rf = RandomForestClassifier()

In [14]:
model_nb = model_nb.fit(X=X_train,y=y_train)
model_dt = model_dt.fit(X=X_train,y=y_train)
model_rf = model_rf.fit(X=X_train,y=y_train)

In [15]:
y_pred_nb = model_nb.predict(X_val)
y_pred_dt = model_dt.predict(X_val)
y_pred_rf = model_rf.predict(X_val)

In [16]:
from sklearn.ensemble import VotingClassifier
ensemble = VotingClassifier(estimators=[('nb', model_nb), ('dt', model_dt), ('rf', model_rf)], voting='hard')
ensemble.fit(X_train, y_train)
y_pred_ensemble = ensemble.predict(X_val)

In [17]:
from sklearn.metrics import accuracy_score, confusion_matrix
print("Naive Bayes Accuracy:", accuracy_score(y_val, y_pred_nb))
print("Naive Bayes Confusion Matrix:\n", confusion_matrix(y_val, y_pred_nb))
print()

print("Decision Tree Accuracy:", accuracy_score(y_val, y_pred_dt))
print("Decision Tree Confusion Matrix:\n", confusion_matrix(y_val, y_pred_dt))
print()

print("Random Forest Accuracy:", accuracy_score(y_val, y_pred_rf))
print("Random Forest Confusion Matrix:\n", confusion_matrix(y_val, y_pred_rf))
print()

print("Ensemble Accuracy:", accuracy_score(y_val, y_pred_ensemble))
print("Ensemble Confusion Matrix:\n", confusion_matrix(y_val, y_pred_ensemble))

Naive Bayes Accuracy: 0.81728
Naive Bayes Confusion Matrix:
 [[2463  622]
 [ 520 2645]]

Decision Tree Accuracy: 0.70512
Decision Tree Confusion Matrix:
 [[2215  870]
 [ 973 2192]]

Random Forest Accuracy: 0.81696
Random Forest Confusion Matrix:
 [[2534  551]
 [ 593 2572]]

Ensemble Accuracy: 0.82384
Ensemble Confusion Matrix:
 [[2519  566]
 [ 535 2630]]


Now with the difficult test set

In [2]:
test_set = [
    "Oh, what a delightful experience! I can't contain my excitement, don't know what I enjoyed more, the rude service or the shocking food.",
    "The worst best restaurant I've ever been to. Terrible food and great service.",
    "Compared to a prison cafeteria, this restaurant is amazing! A true culinary wonderland.",
    "If you're into food that defies the laws of physics, this is the place to be.",
    "I can't say I didn't not dislike the food. It's not terrible, maybe.",
    "Not bad, but not great either. It's not like I'll never visit again, but there's no reason to rush back.",
    "Wow, just wow! This place is an absolute dream, if your dream is a nightmare.",
    "The chef here must have a Michelin-starred grandmother. The food is out of this world, literally.",
    "The ambience, if you can even call it that, adds a certain... charm to the dining experience.",
    "I'll give them points for trying. It's definitely a unique dining experience."
]

In [3]:
labels = [0, 0, 0, 0, 0, 1, 1, 1, 1, 1]

In [19]:
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer='word', max_features=1000, lowercase=True, stop_words='english', ngram_range=(1, 2))
features = vectorizer.fit_transform(test_set).toarray()

model_nb = MultinomialNB()
model_dt = DecisionTreeClassifier()
model_rf = RandomForestClassifier()

model_nb.fit(X=features, y=labels)
model_dt.fit(X=features, y=labels)
model_rf.fit(X=features, y=labels)

y_pred_nb = model_nb.predict(features)
y_pred_dt = model_dt.predict(features)
y_pred_rf = model_rf.predict(features)

ensemble = VotingClassifier(estimators=[('nb', model_nb), ('dt', model_dt), ('rf', model_rf)], voting='hard')

ensemble.fit(features, labels)

y_pred_ensemble = ensemble.predict(features)

print("Naive Bayes Accuracy:", accuracy_score(labels, y_pred_nb))
print("Naive Bayes Confusion Matrix:\n", confusion_matrix(labels, y_pred_nb))
print()

print("Decision Tree Accuracy:", accuracy_score(labels, y_pred_dt))
print("Decision Tree Confusion Matrix:\n", confusion_matrix(labels, y_pred_dt))
print()

print("Random Forest Accuracy:", accuracy_score(labels, y_pred_rf))
print("Random Forest Confusion Matrix:\n", confusion_matrix(labels, y_pred_rf))
print()

print("Ensemble Accuracy:", accuracy_score(labels, y_pred_ensemble))
print("Ensemble Confusion Matrix:\n", confusion_matrix(labels, y_pred_ensemble))



Naive Bayes Accuracy: 1.0
Naive Bayes Confusion Matrix:
 [[5 0]
 [0 5]]

Decision Tree Accuracy: 1.0
Decision Tree Confusion Matrix:
 [[5 0]
 [0 5]]

Random Forest Accuracy: 1.0
Random Forest Confusion Matrix:
 [[5 0]
 [0 5]]

Ensemble Accuracy: 1.0
Ensemble Confusion Matrix:
 [[5 0]
 [0 5]]


Interestingly, all of the accuracy scores were 1 for each of the three models and also for the voting ensemble. This then shows that there is no improvement or decrease in quality when using the ensemble.